[Question Answering Minimal Start](https://simpletransformers.ai/docs/qa-minimal-start/)

In [3]:
%%capture

!pip install simpletransformers

In [2]:
import logging

from simpletransformers.question_answering import QuestionAnsweringModel, QuestionAnsweringArgs

In [3]:
logging.basicConfig(level=logging.INFO)
transformers_logger = logging.getLogger("transformers")
transformers_logger.setLevel(logging.WARNING)

In [4]:
import json
import requests

url = "https://raw.githubusercontent.com/josuemzx/Chatbot-para-refugiados/main/data/Dataset%20propio/data.json"
data = json.loads(requests.get(url).text)

In [5]:
type(data)

dict

In [6]:
list(data.keys())

['data']

In [7]:
type(data["data"])

list

In [8]:
data = [item for topic in data["data"] for item in topic['paragraphs'] ]

In [9]:
from sklearn.model_selection import train_test_split

train_data, eval_data = train_test_split(data, test_size=0.2, random_state=42)

In [10]:
type(train_data[0])

dict

In [11]:
list(train_data[0].keys())

['qas', 'context', 'document_id']

In [12]:
list(train_data[0]["qas"][0])

['question', 'id', 'answers', 'is_impossible']

In [13]:
list(eval_data[0]["qas"][0])

['question', 'id', 'answers', 'is_impossible']

In [14]:
# Configure the model
model_args = QuestionAnsweringArgs()
model_args.train_batch_size = 16
model_args.evaluate_during_training = True

model = QuestionAnsweringModel(
    "roberta", "roberta-base", args=model_args
)

Downloading:   0%|          | 0.00/481 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/501M [00:00<?, ?B/s]

Some weights of the model checkpoint at roberta-base were not used when initializing RobertaForQuestionAnswering: ['lm_head.layer_norm.bias', 'lm_head.bias', 'lm_head.decoder.weight', 'lm_head.layer_norm.weight', 'lm_head.dense.weight', 'lm_head.dense.bias']
- This IS expected if you are initializing RobertaForQuestionAnswering from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForQuestionAnswering from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForQuestionAnswering were not initialized from the model checkpoint at roberta-base and are newly initialized: ['qa_outputs.bias', 'qa_outputs.weight']
You should probably TRAIN this model on a down-stream task to be able to use 

Downloading:   0%|          | 0.00/899k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/456k [00:00<?, ?B/s]

In [15]:
# Train the model
model.train_model(train_data, eval_data=eval_data)

add example index and unique id: 100%|██████████| 84/84 [00:00<00:00, 348143.81it/s]


Epoch:   0%|          | 0/1 [00:00<?, ?it/s]

Running Epoch 0 of 1:   0%|          | 0/9 [00:00<?, ?it/s]

/usr/local/lib/python3.8/dist-packages/torch/optim/lr_scheduler.py:138: UserWarning: Detected call of `lr_scheduler.step()` before `optimizer.step()`. In PyTorch 1.1.0 and later, you should call them in the opposite order: `optimizer.step()` before `lr_scheduler.step()`.  Failure to do this will result in PyTorch skipping the first value of the learning rate schedule. See more details at https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate
  warnings.warn("Detected call of `lr_scheduler.step()` before `optimizer.step()`. "

convert squad examples to features: 100%|██████████| 13/13 [00:00<00:00, 45.08it/s]

add example index and unique id: 100%|██████████| 13/13 [00:00<00:00, 108834.24it/s]


Running Evaluation:   0%|          | 0/3 [00:00<?, ?it/s]

(9,
 {'global_step': [9],
  'correct': [0],
  'similar': [12],
  'incorrect': [1],
  'train_loss': [3.756591796875],
  'eval_loss': [-0.9401041666666666]})

In [16]:
# Evaluate the model
result, texts = model.eval_model(eval_data)

add example index and unique id: 100%|██████████| 13/13 [00:00<00:00, 89240.51it/s]


Running Evaluation:   0%|          | 0/3 [00:00<?, ?it/s]

In [18]:
# Make predictions with the model
to_predict = [
    {
        "context": "Derecho a solicitar protección internacional.\n\n1. Las personas nacionales no comunitarias y las apátridas presentes en territorio español tienen derecho a solicitar protección internacional en España.\n\n2. Para su ejercicio, los solicitantes de protección internacional tendrán derecho a asistencia sanitaria y a asistencia jurídica gratuita, que se extenderá a la formalización de la solicitud y a toda la tramitación del procedimiento, y que se prestará en los términos previstos en la legislación española en esta materia, así como derecho a intérprete en los términos del artículo 22 de la Ley Orgánica 4/2000.\n\nLa asistencia jurídica referida en el párrafo anterior será preceptiva cuando las solicitudes se formalicen de acuerdo al procedimiento señalado en el artículo 21 de la presente Ley.\n\n3. La presentación de la solicitud conllevará la valoración de las circunstancias determinantes del reconocimiento de la condición de refugiado, así como de la concesión de la protección subsidiaria. De este extremo se informará en debida forma al solicitante.\n\n4. Toda información relativa al procedimiento, incluido el hecho de la presentación de la solicitud, tendrá carácter confidencial.",
        "qas": [
            {
                "question": "¿Puedo solicitar traductor?",
                "id": "0",
            }
        ],
    }
]

answers, probabilities = model.predict(to_predict)

print(answers)

add example index and unique id: 100%|██████████| 1/1 [00:00<00:00, 10255.02it/s]


Running Prediction:   0%|          | 0/1 [00:00<?, ?it/s]

[{'id': '0', 'answer': ['.', '', 'español tienen derecho a solicitar protección internacional en España.', 'internacional. 1. Las personas nacionales no comunitarias y las apátridas presentes en territorio español tienen derecho a solicitar protección internacional en España.', 'comunitarias y las apátridas presentes en territorio español tienen derecho a solicitar protección internacional en España.', '1. Las personas nacionales no comunitarias y las apátridas presentes en territorio español tienen derecho a solicitar protección internacional en España.', 'España. 2. Para su ejercicio, los solicitantes de protección internacional tendrán derecho a asistencia sanitaria y a asistencia jurídica gratuita, que se extenderá a la formalización', 'España. 2. Para su ejercicio, los solicitantes de protección internacional tendrán derecho a asistencia sanitaria y a asistencia jurídica gratuita,', 'España. 2. Para su ejercicio, los solicitantes de protección internacional tendrán derecho a asist

In [19]:
to_predict = [
    {
        "context": "Las subvenciones concedidas a las ONG para la acogida y la integración de quienes conforman estos colectivos están financiadas con fondos que provienen de los presupuestos de la Secretaría General de Inmigración y Emigración, recibiendo cofinanciación, en algunos programas, del Fondo Social Europeo y del Fondo de Asilo, Migración e Integración.",
        "qas": [
            {
                "question": "donde solicito las subenciones?",
                "id": "0",
            }
        ],
    }
]

answers, probabilities = model.predict(to_predict)

print(answers)

add example index and unique id: 100%|██████████| 1/1 [00:00<00:00, 10922.67it/s]


Running Prediction:   0%|          | 0/1 [00:00<?, ?it/s]

[{'id': '0', 'answer': ['.', '', 'Emigración, recibiendo cofinanciación, en algunos programas, del Fondo Social Europeo y del Fondo de Asilo, Migración e Integración.', 'programas, del Fondo Social Europeo y del Fondo de Asilo, Migración e Integración.', 'cofinanciación, en algunos programas, del Fondo Social Europeo y del Fondo de Asilo, Migración e Integración.', 'Migración e Integración.', 'Asilo, Migración e Integración.', 'Social Europeo y del Fondo de Asilo, Migración e Integración.', 'Europeo y del Fondo de Asilo, Migración e Integración.', 'y del Fondo de Asilo, Migración e Integración.', 'Las subvenciones concedidas a las ONG', 'ONG', 'G', 'Las', 'Las subvenciones concedidas a las ONG para la acogida y la integración de quienes conforman estos colectivos están financiadas con fondos que provienen de los presupuestos de la Secretaría General de Inmigración y Emigración,', 'Las subvenciones concedidas a las ONG para la acogida y la integración de quienes conforman estos colectiv